In [33]:
def dot_product_rep(G, chi1, chi2):
    """
    Find multiplicity of chi1 in chi2, which are characters of G, assumed to be in the order of G.conjugacy_classes()
    
    Parameters:
    -----------
    G group, chi1, chi2 characters of G
    
    Returns:
    --------
    integer
    """
    classes = list(G.conjugacy_classes())
    class_sizes = [G.order() / G.centralizer(C.representative()).order() for C in classes]
    result = 0
    for idx in range(0,len(chi1)):
        result += class_sizes[idx]*chi1[idx]*chi2[idx]
    return result/G.order()

In [34]:
def induced_rep(H,G,chi):
    """
    Find induced character of chi, which is a character of H, assumed to be in the order of H.conjugacy_classes()
    
    Parameters:
    -----------
    H a subgroup of G, chi a character of H
    
    Returns:
    --------
    character of G
    """
    Gclasses = list(G.conjugacy_classes())
    Hclasses = list(H.conjugacy_classes())
    # use the formula chi(g) = sum_{[h] \subset [g]} C_G(g)/C_H(h) chi(h)
    induced_character = [0]*len(Gclasses)
    for idx_g in range(0,len(G.conjugacy_classes_representatives())):
        g = G.conjugacy_classes_representatives()[idx_g]
        result = 0
        for idx_h in range(0,len(H.conjugacy_classes_representatives())):
            h = H.conjugacy_classes_representatives()[idx_h]
            if h in Gclasses[idx_g]:
                result += (G.centralizer(g).order())/(H.centralizer(h).order())*chi[idx_h]
        induced_character[idx_g] = result
    return induced_character

def restricted_rep(H,G,chi):
    """
    Find restricted character of chi, which is a character of G, assumed to be in the order of G.conjugacy_classes()
    
    Parameters:
    -----------
    H a subgroup of G, chi a character of H
    
    Returns:
    --------
    character of G
    """
    Gclasses = list(G.conjugacy_classes())
    Hclasses = list(H.conjugacy_classes())
    # use the formula chi(g) = sum_{[h] \subset [g]} C_G(g)/C_H(h) chi(h)
    restricted_character = [0]*len(Hclasses)
    for idx_h in range(0,len(H.conjugacy_classes_representatives())):
        h = H.conjugacy_classes_representatives()[idx_h]
        for idx_cls in range(0,len(Gclasses)):
            if h in Gclasses[idx_cls]:
                restricted_character[idx_h] = chi[idx_cls]
                break
    return restricted_character

In [35]:
def compute_partition_coefficient(mu, lam):
    """
    Compute the coefficient of the product of x^(lambda_i + l - i) in the 
    Vandermonde-like product with powers specified by mu.
    
    Args:
        mu (list): First partition (mu_1, ..., mu_m)
        lam (list): Second partition (lambda_1, ..., lambda_l)
    
    Returns:
        The coefficient as a rational number
    """
    # Create the polynomial ring
    n = sum(mu)  # total degree
    l = len(lam)  # length of lambda
    R = PolynomialRing(QQ, 'x', l)
    x = R.gens()
    
    # Construct the Vandermonde factor
    vandermonde = prod(x[i] - x[j] 
                      for j in range(l) 
                      for i in range(j))
    
    # Construct the power sum factor
    power_sums = []
    for power in mu:
        term = sum(x[i]^power for i in range(l))
        power_sums.append(term)
    power_sum_product = prod(power_sums)
    
    # Multiply the factors
    total_polynomial = vandermonde * power_sum_product
    
    # Create the monomial we're looking for
    target_exponents = [0] * n
    for i in range(l):
        target_exponents[i] = lam[i] + l - (i + 1)

    # Get the coefficient
    return total_polynomial.coefficient(target_exponents)

def partition_chartable(n):
    # ordered in ascending permutations, starting at (1,1,...,1)
    G = SymmetricGroup(n)
    partitions = Partitions(n).list()
    return matrix(ZZ, len(partitions), len(partitions), lambda x,y: compute_partition_coefficient(partitions[-y-1],partitions[-x-1]))

#def partition_chartable_ordered(n):
#    G = SymmetricGroup(n)
#    partitions = Partitions(n).list()
#    print(partitions)
#    return matrix(ZZ, len(partitions), len(partitions), lambda x,y: compute_partition_coefficient(partitions[y-1],partitions[x]))

# print(partition_chartable(6))

In [43]:
n = 6
Sn = SymmetricGroup(n)
Sn_minus_1 = Sn.stabilizer(n)
#print(Sn_minus_1.conjugacy_classes_representatives())

In [44]:
# goal is to take each character of S_{n-1}, induce it up to S_n and then decompose it as a sum of irreducibles.
subgroup_chartable = partition_chartable(n-1)
chartable = partition_chartable(n)
for i,char in enumerate(subgroup_chartable):
    # i is index and corresponds to a partition
    part_subgroup = Partitions(n-1).list()[-1-i]
    print("we induce the irrep of S_" + (n-1).str() + " up to S_" + (n).str() + " corresponding to the partition " + ''.join(map(str,part_subgroup)) + ", which decomposes into the sum of")
    induced = induced_rep(Sn_minus_1, Sn, char)
    for j, nchar in enumerate(chartable):
        # j is index and corresponds to a partition
        part = Partitions(n).list()[-1-j]
        # compute how often the row of the chartable appears in it
        multiplicity = dot_product_rep(Sn, induced, nchar)
        if multiplicity > 0:
            print(''.join(map(str,part)) + " (with multiplicity " + multiplicity.str() + ")")

we induce the irrep of S_6 up to S_5 corresponding to the partition 11111, which decomposes into the sum of
111111 (with multiplicity 1)
21111 (with multiplicity 1)
we induce the irrep of S_6 up to S_5 corresponding to the partition 2111, which decomposes into the sum of
21111 (with multiplicity 1)
2211 (with multiplicity 1)
3111 (with multiplicity 1)
we induce the irrep of S_6 up to S_5 corresponding to the partition 221, which decomposes into the sum of
2211 (with multiplicity 1)
222 (with multiplicity 1)
321 (with multiplicity 1)
we induce the irrep of S_6 up to S_5 corresponding to the partition 311, which decomposes into the sum of
3111 (with multiplicity 1)
321 (with multiplicity 1)
411 (with multiplicity 1)
we induce the irrep of S_6 up to S_5 corresponding to the partition 32, which decomposes into the sum of
321 (with multiplicity 1)
33 (with multiplicity 1)
42 (with multiplicity 1)
we induce the irrep of S_6 up to S_5 corresponding to the partition 41, which decomposes into t

In [45]:
# goal is to take each character of S_{n}, restrict it to S_{n-1} and then decompose it as a sum of irreducibles.
subgroup_chartable = partition_chartable(n-1)
chartable = partition_chartable(n)
for i,char in enumerate(chartable):
    # i is index and corresponds to a partition
    part = Partitions(n).list()[-1-i]
    print("we restrict the irrep of S_" + (n).str() + " down to S_" + (n-1).str() + " corresponding to the partition " + ''.join(map(str,part)) + ", which decomposes into the sum of")
    restricted = restricted_rep(Sn_minus_1, Sn, char)
    for j, subgroup_char in enumerate(subgroup_chartable):
        # j is index and corresponds to a partition
        part_subgroup = Partitions(n-1).list()[-1-j]
        multiplicity = dot_product_rep(Sn_minus_1, restricted, subgroup_char)
        if multiplicity > 0:
            print(''.join(map(str,part_subgroup)) + " (with multiplicity " + multiplicity.str() + ")")

we restrict the irrep of S_6 down to S_5 corresponding to the partition 111111, which decomposes into the sum of
11111 (with multiplicity 1)
we restrict the irrep of S_6 down to S_5 corresponding to the partition 21111, which decomposes into the sum of
11111 (with multiplicity 1)
2111 (with multiplicity 1)
we restrict the irrep of S_6 down to S_5 corresponding to the partition 2211, which decomposes into the sum of
2111 (with multiplicity 1)
221 (with multiplicity 1)
we restrict the irrep of S_6 down to S_5 corresponding to the partition 222, which decomposes into the sum of
221 (with multiplicity 1)
we restrict the irrep of S_6 down to S_5 corresponding to the partition 3111, which decomposes into the sum of
2111 (with multiplicity 1)
311 (with multiplicity 1)
we restrict the irrep of S_6 down to S_5 corresponding to the partition 321, which decomposes into the sum of
221 (with multiplicity 1)
311 (with multiplicity 1)
32 (with multiplicity 1)
we restrict the irrep of S_6 down to S_5 